In [ ]:
import featuretools as ft

In [ ]:
train_set_df = pd.read_csv("C:/Users/srinivasa.jami/.kaggle/competitions/home-credit-default-risk/application_train.csv", dtype = main_features_dtypes_dict)
test_set_df = pd.read_csv("C:/Users/srinivasa.jami/.kaggle/competitions/home-credit-default-risk/application_test.csv", dtype = main_features_dtypes_dict)

In [ ]:
test_set_df['set'] = 'test'
test_set_df['TARGET'] = -999
train_set_df['set'] = 'train'
app = train_set_df.append(test_set_df, ignore_index = True)

In [ ]:
es = ft.EntitySet(id = 'applications')

es = es.entity_from_dataframe(entity_id = 'app', dataframe = app, index = 'SK_ID_CURR')
es = es.entity_from_dataframe(entity_id = 'bureau', dataframe = bureau_data_df, index = 'SK_ID_BUREAU')
es = es.entity_from_dataframe(entity_id = 'previous_app', dataframe = previous_application_data_df, index = 'SK_ID_PREV')

es = es.entity_from_dataframe(entity_id = 'bureau_balance', dataframe = bureau_balance_data_df, 
                              make_index = True, index = 'bb_index')
es = es.entity_from_dataframe(entity_id = 'cash', dataframe = pos_cash_balance_data_df, 
                              make_index = True, index = 'cash_index')
es = es.entity_from_dataframe(entity_id = 'installments', dataframe = installments_payments_data_df,
                              make_index = True, index = 'in_index')
es = es.entity_from_dataframe(entity_id = 'credit', dataframe = credit_card_balance_data_df,
                              make_index = True, index = 'credit_index')


# Relationship between app and bureau
r_app_bureau = ft.Relationship(es['app']['SK_ID_CURR'], es['bureau']['SK_ID_CURR'])

# Relationship between bureau and bureau balance
r_bureau_balance = ft.Relationship(es['bureau']['SK_ID_BUREAU'], es['bureau_balance']['SK_ID_BUREAU'])

# Relationship between current app and previous apps
r_app_previous = ft.Relationship(es['app']['SK_ID_CURR'], es['previous_app']['SK_ID_CURR'])

# Relationships between previous apps and cash, installments, and credit
r_previous_cash = ft.Relationship(es['previous_app']['SK_ID_PREV'], es['cash']['SK_ID_PREV'])
r_previous_installments = ft.Relationship(es['previous_app']['SK_ID_PREV'], es['installments']['SK_ID_PREV'])
r_previous_credit = ft.Relationship(es['previous_app']['SK_ID_PREV'], es['credit']['SK_ID_PREV'])

In [ ]:
es = es.add_relationships([r_app_bureau, r_bureau_balance, r_app_previous,
                           r_previous_cash, r_previous_installments, r_previous_credit])

In [ ]:
%%time
feature_matrix, feature_names = ft.dfs(entityset=es, target_entity = 'app', max_depth=2)